In [ ]:
import psana as ps
import numpy as np
import matplotlib.pyplot as plt



def get_trace_run_mean(ds_run, tof_range=[35000, 50000], gmd_range=[0,np.inf], xgmd_range=[0,np.inf], nmax=100):
    trc = np.zeros(tof_range[1]-tof_range[0])
    n_trc = 0

    for i_evt, evt in enumerate(ds_run.events()):

        times, trace = get_trace(evt)
        if trace is None:
            continue
        gmd = evt.run().Detector('gmd').raw.energy(evt)
        if gmd is None or gmd < gmd_range[0] or gmd > gmd_range[1]:
            continue
        xgmd = evt.run().Detector('xgmd').raw.energy(evt)
        if xgmd is None or xgmd < xgmd_range[0] or xgmd > xgmd_range[1]:
            continue

        t = times[tof_range[0]:tof_range[1]]
        x = trace[tof_range[0]:tof_range[1]]

        x -= np.mean(x[100:700])

        trc = ( trc * (n_trc) + x ) / (n_trc+1)
        n_trc += 1

        if n_trc > nmax:
            break

    return t, trc


def get_trace(evt, tof_channel=None, ranges_bg=[0, 10000], n_digitizers_per_channel=4):
    '''
    Parameters:
        evt : element of ds_run.events()
            psana evt object
        tof_channel : int, optional
            ADC channel
        ranges_bg : 2 element array, optional
            element range for offset subtraction
        n_digitizers_per_channel : int, optional
             Number of digitizers per tof_channel. Relevant for subtracting offset
             from each of the digitizers independently. Is typically a power of 2.
    Returns:
        tof_trace : array
            ToF trace with subtracted offset.
        tof_times : array
            ToF times

    Anatoli Ulmer, 2022
    '''
    hsd = evt.run().Detector('hsd')
    hsd_data = hsd.raw.waveforms(evt)

    if hsd_data is None:
        return None, None

    if tof_channel is None:
        # If no channel is specified and only one hsd channel is active, it will be chosen.
        # If multiple channels are active, the default channel 3 will be chosen.
        if len(hsd_data.keys()) == 1:
            tof_channel = list(hsd_data.keys())[0]  # the digitzer channel the tof is on
        else:
            tof_channel = 3  # the default is channel 3

    tof_data = hsd_data[tof_channel]

    if tof_data is None:
        return None, None

    tof_times = tof_data['times']
    tof_trace = np.asarray(tof_data[0], dtype=float)  # the actual tof data
    tof_trace = bg_correction(tof_trace, ranges_bg=ranges_bg, nchannels=n_digitizers_per_channel)

    # convert to mV
    _hsd_fs_range_vpp = hsd.raw._seg_configs()[tof_channel].config.expert.fs_range_vpp
    tof_trace *= _hsd_to_mv(_hsd_fs_range_vpp)
    return tof_times, tof_trace


def bg_correction(trace, ranges_bg=[0, 10000], nchannels=4):
    for i in np.arange(nchannels):
        bg = np.mean(trace[i + ranges_bg[0]:ranges_bg[1]:nchannels])
        trace[i::nchannels] = trace[i::nchannels] - bg
    return trace


def _hsd_to_mv(_hsd_fs_range_vpp):
    return (400 + _hsd_fs_range_vpp * (1040 - 480) / (65535 - 8192)) / 4096


In [ ]:


# first get gmd/xgmd to get ranges

# TOF voltages: [repeller, extractor, MCP front, Scintillator, MCP back, PMT]
# TOF voltages xleap: [1000, -1000, -950, 2200, 0, -700]
# TOF voltages sase: [1000, -1000, -900, 1500, 0, 650]

# photon energies: [eLog, calibrated]
# xleap: [911 eV, 898 eV]
# sase: [885 eV, 872 eV]
# sase2: [828 eV, 815 eV]

run_dict = {"xleap": [149, 150, 151, 152, 153, 154],
            "sase": [350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360],
            "sase2": [492]}

for ftype, run_list in run_dict.items():

    # plt.figure(figsize=(10,8))
    fig, ax = plt.subplots(2,1,figsize=(8,10))

    for i_run, run in enumerate(run_list):

        ds = ps.DataSource(exp='tmolv1720', run=run,
                           detectors=['gmd', 'xgmd'])
        ds_run = next(ds.runs())

        gmd_array = np.array([])
        xgmd_array = np.array([])

        for i_evt, evt in enumerate(ds_run.events()):

            gmd = evt.run().Detector('gmd').raw.energy(evt)
            if gmd is None:
                continue
            xgmd = evt.run().Detector('xgmd').raw.energy(evt)
            if xgmd is None:
                continue

            gmd_array = np.append(gmd_array, gmd)
            xgmd_array = np.append(xgmd_array, xgmd)

            if i_evt > 1000:
                break


        ax[0].hist(gmd_array, density=True, alpha=.5,
                 label=f"run{run}, gmd={np.mean(gmd_array):.3f}" +
                       f" +- {np.std(gmd_array):.3f}")
        ax[1].hist(xgmd_array, density=True, alpha=.5,
                 label=f"run{run}, xgmd={np.mean(xgmd_array):.3f}"
                       f" +- {np.std(xgmd_array):.3f}")

    ax[0].legend(title=ftype)
    ax[1].legend(title=ftype)


In [ ]:
import psana as ps
import numpy as np
import matplotlib.pyplot as plt


tof_range = [35000, 45000]
t0 = 6.028054e-6
show_charge_states = False

run_dict = {"xleap": [149, 150, 151, 152, 153, 154],
            "sase": [350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360],
            "sase2": [492]}

gmd_range_dict = {"xleap": [0.05, 0.2], "sase": [1.2, 1.8], "sase2": [1.2, 1.8]}
xgmd_range_dict = {"xleap": [0.05, 0.15], "sase": [0.35, 0.5], "sase2": [.85, 1]}


for ftype, run_list in run_dict.items():
    gmd_range = gmd_range_dict[ftype]
    xgmd_range = xgmd_range_dict[ftype]

    plt.figure(figsize=(16,10))

    for i_run, run in enumerate(run_list):

        ds = ps.DataSource(exp='tmolv1720', run=run,
                           detectors=['gmd', 'xgmd', 'hsd'])
        ds_run = next(ds.runs())

        t, trc = get_trace_run_mean(ds_run, tof_range=tof_range, nmax=1000,
                            xgmd_range=xgmd_range)
        t -= t0
        plt.plot(t, trc, label=f"run{run:}")


    if show_charge_states:
        m = 0.220733e-6
        b = 0
        for i in range(1, 18):
            plt.axvline(x=(m * np.sqrt(39.948 / i) + b), color='red', ymin=-40, ymax=0,
                        linestyle='--')
    plt.grid(1)
    plt.legend()
    plt.show()

In [ ]:
# show only optimum position

import psana as ps
import numpy as np
import matplotlib.pyplot as plt


tof_range = [35000, 45000]
t0 = 6.028054e-6
show_charge_states = True

run_dict = {"xleap": [151], "sase": [355]}
gmd_range_dict = {"xleap": [0.05, 0.2], "sase": [1.2, 1.8]}
xgmd_range_dict = {"xleap": [0.05, 0.15], "sase": [0.35, 0.5]}


for ftype, run_list in run_dict.items():
    gmd_range = gmd_range_dict[ftype]
    xgmd_range = xgmd_range_dict[ftype]

    plt.figure(figsize=(16,10))

    for i_run, run in enumerate(run_list):

        ds = ps.DataSource(exp='tmolv1720', run=run,
                           detectors=['gmd', 'xgmd', 'hsd'])
        ds_run = next(ds.runs())

        t, trc = get_trace_run_mean(ds_run, tof_range=tof_range, nmax=1000,
                            xgmd_range=xgmd_range)
        t -= t0
        plt.plot(t, trc, label=f"run{run:}")


    if show_charge_states:
        mq_slope = 0.220733e-6
        t_offset = 0
        for q in np.arange(1, 19):
            plt.axvline(x=(mq_slope * np.sqrt(40 / q) + t_offset), color='red', ymin=-40, ymax=15,
                        linestyle='--', label='')
        plt.axvline(x=(mq_slope * np.sqrt(40 / 1) + t_offset), color='red', ymin=-40, ymax=15,
                        linestyle='--', label='Ar')
        plt.axvline(x=(mq_slope * np.sqrt(14/7) + t_offset), color='orange', ymin=-40, ymax=15,
            linestyle='--', label=f'N$^{{7+}}$')
        plt.axvline(x=(mq_slope * np.sqrt(16/8) + t_offset), color='lightgreen', ymin=-40, ymax=15,
            linestyle='--', label=f'O$^{{8+}}$')
        plt.axvline(x=(mq_slope * np.sqrt(17/9) + t_offset), color='limegreen', ymin=-40, ymax=15,
            linestyle='--', label=f'OH$^{{9+}}$')
        plt.axvline(x=(mq_slope * np.sqrt(18/10) + t_offset), color='darkgreen', ymin=-40, ymax=15,
            linestyle='--', label=f'H$_2$O$^{{10+}}$')
        plt.axvline(x=(mq_slope * np.sqrt(1) + t_offset), color='yellow', ymin=-40, ymax=15,
            linestyle='--', label='H')
    plt.grid(1)
    plt.legend()
    plt.show()
